In [ ]:
using Pkg
Pkg.add("Optim")
Pkg.add("Roots")
Pkg.add("NPZ")
Pkg.add("Interpolations")
Pkg.add("Distributed")

using Optim
using Roots
using NPZ
using Distributed

In [ ]:
gamma = 8.0
rho = 1.00001
fraction = 0.0001
Delta = 1000
dataname = "local"

In [ ]:

symmetric_returns    = 1
state_dependent_xi   = 0
optimize_over_ell    = 0
compute_irfs         = 0                    # need to start julia with "-p 5"

if compute_irfs == 1
    @everywhere include("newsets_utils_modif.jl")
elseif compute_irfs ==0
    include("newsets_utils_modif.jl")
end

println("=============================================================")
if symmetric_returns == 1
    println(" Economy with two capital stocks: SYMMETRIC RETURNS          ")
    if state_dependent_xi == 0
        println(" No tilting (xi is NOT state dependent)                      ")
        filename = (compute_irfs==0) ? "model_sym_HS.npz" : "model_sym_HS_p.npz";
    elseif state_dependent_xi == 1
        println(" With tilting (change in kappa)                        ")
        filename = (compute_irfs==0) ? "model_sym_HSHS.npz" : "model_sym_HSHS_p.npz";
    elseif state_dependent_xi == 2
        println(" With tilting (change in beta)                        ")
        filename = (compute_irfs==0) ? "model_sym_HSHS2.npz" : "model_sym_HSHS2_p.npz";
    end
elseif symmetric_returns == 0
    println(" Economy with two capital stocks: ASYMMETRIC RETURNS         ")
    if state_dependent_xi == 0
        println(" No tilting (xi is NOT state dependent)                      ")
        filename = (compute_irfs==0) ? "model_asym_HS.npz" : "model_asym_HS_p.npz";
    elseif state_dependent_xi == 1
        println(" With tilting (change in kappa)                        ")
        filename = (compute_irfs==0) ? "model_asym_HSHS.npz" : "model_asym_HSHS_p.npz";
    elseif state_dependent_xi == 2
        println(" With tilting (change in beta)                        ")
        filename = (compute_irfs==0) ? "model_asym_HSHS2.npz" : "model_asym_HSHS2_p.npz";
    end
end

filename_ell = "./output/"*dataname*"/"

In [ ]:

# (1) Baseline model
a11 = 0.014
alpha = 0.05
zeta = 0.5
kappa = 0.0

scale = 1.32
sigma_k1 = scale*[.0048,               .0,   .0];
sigma_k2 = scale*[.0              , .0048,   .0];
sigma_z =  [.011*sqrt(.5)   , .011*sqrt(.5)   , .025];

eta1 = 0.013
eta2 = 0.013
beta1 = 0.01
beta2 = 0.01

delta = .002;

phi1 = 28.0
phi2 = 28.0

# (3) GRID
II, JJ = 501, 201;
rmax =  log(20);
rmin = -log(20); 
zmax = 0.75;
zmin = -zmax;

# (4) Iteration parameters
maxit = 10000;        # maximum number of iterations in the HJB loop
crit  = 10e-6;      # criterion HJB loop
# Delta = 1000.;      # delta in HJB algorithm


# Initialize model objects -----------------------------------------------------
baseline1 = Baseline(a11, zeta, kappa, sigma_z, beta1, eta1, sigma_k1, delta);
baseline2 = Baseline(a11, zeta, kappa, sigma_z, beta2, eta2, sigma_k2, delta);
technology1 = Technology(alpha, phi1);
technology2 = Technology(alpha, phi2);
model = TwoCapitalEconomy(baseline1, baseline2, technology1, technology2);

grid = Grid_rz(rmin, rmax, II, zmin, zmax, JJ);
params = FinDiffMethod(maxit, crit, Delta);

#==============================================================================#
# WITH ROBUSTNESS
#==============================================================================#

println(" (3) Compute value function WITH ROBUSTNESS")
A, V, val, d1_F, d2_F, d1_B, d2_B, h1_F, h2_F, hz_F, h1_B, h2_B, hz_B,
        mu_1_F, mu_1_B, mu_r_F, mu_r_B, mu_z, V0, rr, zz, pii, dr, dz =
        value_function_twocapitals(gamma, rho, fraction, model, grid, params, symmetric_returns);
println("=============================================================")
